In [2]:
import numpy as np
from numpy.linalg import eig
from numpy.linalg import inv,pinv
import pandas as pd
import csv
from collections import defaultdict
from functools import partial
import matplotlib.cm as cm
import matplotlib.pyplot as plt
from IPython.display import display
import math
%run common_functions.ipynb
%run naive_bayes_classifier.ipynb

In [3]:
class LogisticRegression:
    def __init__(self, lr=0.1, max_iter=5000, cost_thresold = 0.001, fit_intercept=True, verbose=False):
        self.lr = lr
        self.max_iter = max_iter
        self.fit_intercept = fit_intercept
        self.verbose = verbose
        self.cost_thresold = cost_thresold
    
    def add_intercept(self, X):
        intercept = np.ones((X.shape[0], 1))
        return np.concatenate((intercept, X), axis=1)
    
    def getZ(self, x, theta):
        return np.dot(x, theta.T)
    
    
    def sigmoid(self, z):
        return 1/(1+np.exp(-z))
    
    
    def getCost(self, theta, X, y):
        m = len(y)
        sigma = self.sigmoid(self.getZ(X, theta))
        positive_loss = np.multiply(y, np.log(sigma))
        negative_loss = np.multiply(1 - y, np.log(1 - sigma))
        return -np.mean(positive_loss + negative_loss)


    def getGradients(self, theta, X, y):
        m = len(y)
        temp = self.sigmoid(self.getZ(X, theta)) - y
        return (1 / m) * np.dot(X.T, temp)
    
    
    def gradientDescent(self, class_index, X, y):
        class_theta = self.theta[class_index]
        costs = []

        for iteration in range(self.max_iter):
            currentGradient = self.getGradients(class_theta, X, y)
            currentCost = self.getCost(class_theta, X, y)
            class_theta -= self.lr * currentGradient
            costs.append(currentCost)
            
            if(self.verbose == True and iteration % 1000 == 0):
                print(f'loss: {self.getCost(class_theta, X, y)} \t')
                
            if abs(currentCost) < self.cost_thresold:
                if(self.verbose == True):
                    print("loop broke by thresold")
                break

        return costs, class_theta
    
    
    def fit(self, X, y):
        if self.fit_intercept:
            X = self.add_intercept(X)
            
        (m, n) = X.shape
        self.classes = np.unique(y)
        k = len(self.classes)
        self.theta = np.zeros((k, n)) #weights initialization
        class_costs = {}

#         Initial call to print 0% progress
        printProgressBar(0, k, prefix = 'Logistic Regression Progress:', suffix = 'Complete', length = 50)
        
        for class_index in range(k):
            printProgressBar(class_index + 1, k, prefix = 'Logistic Regression Progress:', suffix = 'Complete', length = 50)
            if(self.verbose == True):
                print(self.classes[class_index])
            binary_class = (y == class_index).flatten()
            costs, self.theta[class_index] = self.gradientDescent(class_index, X, binary_class)
            class_costs[class_index] = costs
            
        return class_costs

    
    def predict(self, X, y):
        if self.fit_intercept:
            X = self.add_intercept(X)

#         predictions = self.classes[np.argmax(X @ self.theta.T, axis = 1)]
        all_z = self.getZ(X, self.theta)
        predictions = self.classes[np.argmax(self.sigmoid(all_z), axis = 1)]
        accuracy = np.mean(predictions == y.flatten()) * 100
        
        return accuracy, predictions   